# 하이퍼 매개 변수 튜닝

다수의 기계 학습 알고리즘에는 *하이퍼 매개 변수*가 필요합니다. 하이퍼 매개 변수란 학습에는 영향을 주지만 학습 데이터 자체에서는 확인할 수 없는 매개 변수 값입니다. 예를 들어 로지스틱 회귀 모델 학습 시에는 *정규화 비율* 하이퍼 매개 변수를 사용하여 모델의 바이어스를 완화할 수 있습니다. 그리고 콘볼루션 신경망 학습 시에는 *학습 속도*, *일괄 처리 크기* 등의 하이퍼 매개 변수를 사용하여 가중치 조정 방식과 미니 배치에서 처리되는 데이터 항목의 수를 각각 제어할 수 있습니다. 선택하는 하이퍼 매개 변수 값에 따라 학습된 모델의 성능이나 모델을 학습시키는 데 걸리는 시간이 크게 달라질 수 있으며, 여러 매개 변수 조합을 적용하여 최적의 값을 찾아야 하는 경우가 많습니다.

여기서는 하이퍼 매개 변수가 하나인 간단한 로지스틱 회귀 모델의 예제를 사용할 것입니다. 하지만 이 모델의 원칙은 Azure Machine Learning을 사용하여 학습시킬 수 있는 모든 종류의 모델에 적용할 수 있습니다.

## 작업 영역에 연결

가장 먼저 해야 하는 작업은 Azure ML SDK를 사용하여 작업 영역에 연결하는 것입니다.

> **참고**: 이전 연습을 완료한 후 Azure 구독으로 인증된 세션이 만료된 경우 다시 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 실험용 데이터 준비

이 랩에서는 당뇨병 환자의 세부 정보가 포함된 데이터 세트를 사용합니다. 아래 셀을 실행하여 이 데이터 세트를 만듭니다. 이전 랩에서 해당 데이터 세트를 만든 경우 코드를 실행하면 새 버전이 작성됩니다.

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # /data에서 당뇨병 CSV 파일 업로드
                        target_path='diabetes-data/', # 데이터 저장소의 폴더 경로에 해당 파일 저장
                        overwrite=True, # 이름이 같은 기존 파일 바꾸기
                        show_progress=True)

    #데이터 저장소의 경로에서 테이블 형식 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 테이블 형식 데이터 세트 등록
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## 학습 스크립트 준비

먼저 로지스틱 회귀 모델을 학습시키는 데 사용할 학습 스크립트용 폴더를 만들어 보겠습니다.

In [ ]:
import os

experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

이제 모델 학습을 위한 Python 스크립트를 만듭니다. 이 스크립트에는 다음 항목이 포함되어야 합니다.

- 최적화할 각 하이퍼 매개 변수(여기서는 정규화 하이퍼 매개 변수뿐임)에 해당하는 매개 변수
- 최적화할 성능 메트릭을 기록하는 코드. 여기서는 AUC와 정확도를 모두 기록하므로, AUC와 정확도 중 하나를 최대한 높이도록 모델을 최적화할 수 있습니다.

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import argparse
import joblib
import os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 정규화 매개 변수 설정
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='input_data', help='training dataset')
args = parser.parse_args()
reg = args.reg_rate

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 세트 로드
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe() # 추정기 입력에서 학습 데이터 가져오기

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 컴퓨팅 대상 준비

클라우드 컴퓨팅의 이점 중 하나는 요청 시에 크기를 조정할 수 있다는 것입니다. 따라서 하이퍼 매개 변수 값을 각기 다르게 설정하여 한 실험의 여러 실행을 병렬로 처리하기에 충분한 컴퓨팅 리소스를 프로비전할 수 있습니다.

이전 랩에서 만든 Azure Machine Learning 컴퓨팅 클러스터를 사용합니다. 이 클러스터는 아직 없으면 자동으로 생성됩니다.

> **중요**: 계산 클러스터를 실행하기 전에 *your-compute-cluster*를 아래 코드에서 컴퓨팅 클러스터의 이름으로 변경하세요! 클러스터 이름은 길이가 2~16자 사이의 전역으로 고유한 이름이어야 합니다. 유효한 문자는 문자, 숫자 및 문자입니다.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # 기존 컴퓨팅 대상 확인
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # 아직 존재하지 않는 경우, 생성합니다.
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

## *하이퍼 드라이브* 실험 실행

Azure Machine Learning에는 *하이퍼 드라이브* 실험을 통한 하이퍼 매개 변수 튜닝 기능이 포함되어 있습니다. 이러한 실험에서는 하이퍼 매개 변수 조합이 각기 다른 여러 하위 실행이 시작됩니다. 따라서 성능이 가장 우수한 모델을 생성하는 실행을 확인한 다음 이 실행에서 학습된 모델을 등록 및 배포용으로 선택할 수 있습니다. 여기서 성능이 가장 우수한 모델은 기록된 대상 성능 메트릭 중 최적화하려는 메트릭을 기준으로 결정됩니다.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# 실험용 Python 환경 만들기
sklearn_env = Environment("sklearn-env")

# 필수 패키지가 설치되어 있는지 확인합니다. scikit-learn, Azure ML defaults 및 Azure ML dataprep 패키지가 필요합니다.
packages = CondaDependencies.create(pip_packages=['scikit-learn','azureml-defaults','azureml-dataprep[pandas]'])
sklearn_env.python.conda_dependencies = packages

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--regularization', 0.1, # 정규화율 매개 변수
                                           '--input-data', diabetes_ds.as_named_input('training_data')], # 데이터 세트 참조
                              environment=sklearn_env,
                              compute_target = training_cluster)

# 매개 변수 값 범위 샘플링
params = GridParameterSampling(
    {
        # 매개 변수가 하나뿐이므로 표 형태 샘플링에서는 각 값을 적용해 봅니다. 매개 변수가 여러 개일 때는 모든 조합을 적용해 봅니다.
        '--regularization': choice(0.001, 0.005, 0.01, 0.05, 0.1, 1.0)
    }
)

# 하이퍼 드라이브 설정 구성
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, 
                          primary_metric_name='AUC', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6,
                          max_concurrent_runs=4)

# 실험 실행
experiment = Experiment(workspace = ws, name = 'diabates_training_hyperdrive')
run = experiment.submit(config=hyperdrive)

# 실험이 실행될 때 Notebook의 상태 표시
RunDetails(run).show()
run.wait_for_completion()

위의 위젯에서 실험 실행 상태를 확인할 수 있습니다. [Azure Machine Learning Studio](https://ml.azure.com)에서 기본 하이퍼 드라이브 실험 실행 및 해당 하위 실험을 확인할 수도 있습니다.

> **참고**: 위젯이 새로 고침되지 않을 수 있습니다. 실행이 완료되면 위젯 아래에 요약 정보가 표시됩니다.

## 성능이 가장 우수한 실행 결정

모든 실행이 완료되면 지정한 성능 메트릭을 기준으로 하여 성능이 가장 우수한 실행(여기서는 AUC가 가장 높은 실행)을 찾을 수 있습니다.

In [ ]:
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Regularization Rate:',parameter_values)

성능이 가장 우수한 실행을 찾은 후에는 이 실행에서 학습된 모델을 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

# 모델 등록
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# 등록된 모델 목록 표시
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **추가 정보**: 하이퍼 드라이브에 대한 자세한 내용은 [Azure ML 설명서](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters)를 참조하세요.